In [1]:
#IMPORTS

import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException

import requests
from bs4 import BeautifulSoup
import time

In [2]:
df = pd.DataFrame(columns=['href_list']) # creates master dataframe 

# Chromedriver Path
PATH = "/Users/javm/Desktop/chromedriver_mac_arm64/chromedriver"

# this creates/opens Chrome browser
driver = webdriver.Chrome(PATH)

#creating list of href
href_list = []

for listings in range(1,20):
    page_num = str(listings)
    url = 'https://www.barnesandnoble.com/b/books/_/N-1z13w9sZ1sZ29Z8q8?Nrpp=20&page=' + page_num
    driver.get(url)
    # freeze/halt python so url loads completely
    driver.implicitly_wait(60)
    
    #find hrefs
    div_elements = driver.find_elements(By.CSS_SELECTOR, "div.product-shelf-title.product-info-title.pt-xs")
    for div_element in div_elements:
        elements = div_element.find_elements(By.TAG_NAME, "a")
        for element in elements:
            href = element.get_attribute('href')
            href_list.append(href)

    urls_df = pd.DataFrame(href_list, columns=['href_list']) # creates master dataframe
    df = pd.concat([df, urls_df], ignore_index=True)
    
driver.close()


/var/folders/6g/zn0bw_6s1ws2qwbb44k4ppkw0000gn/T/ipykernel_19295/3821222310.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [3]:
len(href_list)

380

In [4]:
# Chromedriver Path
PATH = "/Users/javm/Desktop/chromedriver_mac_arm64/chromedriver"

# this creates/opens Chrome browser
driver = webdriver.Chrome(PATH)

book_details = []

for index, href in enumerate(href_list):
    driver.get(href)
    driver.implicitly_wait(60)

    # find titles
    titles = driver.find_elements(By.TAG_NAME, 'h1')
    title_list = []
    for t in range(len(titles)):
        title_list.append(titles[t].text)

    if not title_list:
        continue

    # find author
    author_list = []
    try:
        author_elems = driver.find_elements(By.CSS_SELECTOR, '.contributors.pdp-book-author a')
        for author_elem in author_elems:
            author_list.append(author_elem.text)
    except NoSuchElementException:
        author_list = [np.nan]

    # find publisher
    try:
        publisher_span = driver.find_element(By.CSS_SELECTOR, 'span[itemprop="publisher"]')
        publisher = publisher_span.text
    except NoSuchElementException:
        publisher = np.nan

    # find blurb
    try:
        parent_div = driver.find_element(By.XPATH, '//*[@id="overviewSection"]')
        blurb_text = parent_div.text
    except NoSuchElementException:
        blurb_text = np.nan
    
    book = [title_list[1] if len(title_list) > 1 else title_list[0], author_list, publisher, blurb_text]
    book_details.append(book)

driver.quit()

BOOK_DF_1 = pd.DataFrame(book_details, columns=['title', 'authors', 'publisher', 'blurb'])



/var/folders/6g/zn0bw_6s1ws2qwbb44k4ppkw0000gn/T/ipykernel_19295/1683694768.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [6]:
BOOK_DF_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      380 non-null    object
 1   authors    380 non-null    object
 2   publisher  380 non-null    object
 3   blurb      377 non-null    object
dtypes: object(4)
memory usage: 12.0+ KB


In [7]:
BOOK_DF_1.to_csv('books_1', index = False)